In [66]:
import pandas as pd

In [64]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
import gc #内存管理
file_dir=r'D:\bigdata_project\kaggle_dataset\elo-merchant-category-recommendation'
os.chdir(file_dir)
os.listdir(file_dir)
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
from tqdm import tqdm
tqdm.pandas()

# 读取表

In [2]:
train=pd.read_csv(os.path.join(file_dir,'train.csv'))
test=pd.read_csv(os.path.join(file_dir,'test.csv'))
merchant_info=pd.read_csv(file_dir+'\\merchants.csv')
new_merchant_transactions=pd.read_csv(file_dir+'\\new_merchant_transactions.csv')
historical_transactions=pd.read_csv(file_dir+'\\historical_transactions.csv')

# 训练与测试表预处理

In [3]:
first_active_month=list(set(train['first_active_month'].tolist()+test['first_active_month'].tolist()))
first_active_month_le=LabelEncoder()
first_active_month_le.fit(first_active_month)
train['first_active_month_encod']=first_active_month_le.transform(train['first_active_month'])
test['first_active_month_encod']=first_active_month_le.transform(test['first_active_month'])
train.to_csv('train_pre.csv',index=False)
test.to_csv('test_pre.csv',index=False)
del train
del test

In [4]:
gc.collect()

0

# merchant表预处理

In [5]:
# 拆分离散变量和数值变量
merchant_numerical_feature=['numerical_1', 'numerical_2',
       'active_months_lag3', 'active_months_lag6','active_months_lag12',
       'avg_sales_lag3', 'avg_purchases_lag3',
       'avg_sales_lag6', 'avg_purchases_lag6',
       'avg_sales_lag12', 'avg_purchases_lag12']
merchant_category_feature=['merchant_id','merchant_group_id', 'merchant_category_id','subsector_id','category_1','most_recent_sales_range', 'most_recent_purchases_range', 'category_4', 'city_id', 'state_id', 'category_2']

In [6]:
multi_le=OrdinalEncoder()
merchant_info[['category_1', 'most_recent_sales_range','most_recent_purchases_range', 'category_4']]=multi_le.fit_transform(merchant_info[['category_1', 'most_recent_sales_range','most_recent_purchases_range', 'category_4']])

In [7]:
merchant_info.fillna(value=-1, inplace=True)

In [8]:
len(merchant_info.describe().columns)+1==len(merchant_info.columns)

True

In [9]:
inf_cols=['avg_purchases_lag3','avg_purchases_lag6','avg_purchases_lag12']
for col in inf_cols:
       true_max=max(merchant_info[col].replace(to_replace=np.inf,value=-999999))
       merchant_info[col]=merchant_info[col].replace(to_replace=np.inf,value=true_max)
       mid_value=merchant_info[col].quantile(0.5)
       merchant_info[col].fillna(value=mid_value, inplace=True)
merchant_info.describe()

,merchant_group_id,merchant_category_id,subsector_id,numerical_1,numerical_2,category_1,most_recent_sales_range,most_recent_purchases_range,avg_sales_lag3,avg_purchases_lag3,...,avg_sales_lag6,avg_purchases_lag6,active_months_lag6,avg_sales_lag12,avg_purchases_lag12,active_months_lag12,category_4,city_id,state_id,category_2
count,334696.000000,334696.000000,334696.000000,334696.000000,334696.000000,334696.000000,334696.000000,334696.000000,334696.000000,334696.000000,...,3.346960e+05,334696.000000,334696.000000,3.346960e+05,334696.000000,334696.000000,334696.000000,334696.000000,334696.000000,334696.000000
mean,31028.736143,423.131663,25.116404,0.011476,0.008103,0.021031,3.388233,3.382565,13.832416,2.145143,...,2.164991e+01,2.390194,5.947397,2.522669e+01,2.529277,11.599335,0.287126,102.917926,11.860942,2.259958
std,31623.043426,252.898046,9.807371,1.098154,1.070497,0.143488,0.753297,0.752935,2395.443478,213.955844,...,3.947031e+03,194.324264,0.394936,5.251740e+03,174.419034,1.520138,0.452422,107.090673,6.176889,1.657263
min,1.000000,-1.000000,-1.000000,-0.057471,-0.057471,0.000000,0.000000,0.000000,-82.130000,0.333495,...,-8.213000e+01,0.167045,1.000000,-8.213000e+01,0.098330,1.000000,0.000000,-1.000000,-1.000000,-1.000000
25%,3612.000000,222.000000,19.000000,-0.057471,-0.057471,0.000000,3.000000,3.000000,0.880000,0.923650,...,8.500000e-01,0.902247,6.000000,8.500000e-01,0.898333,12.000000,0.000000,-1.000000,9.000000,1.000000
50%,19900.000000,373.000000,27.000000,-0.057471,-0.057471,0.000000,4.000000,4.000000,1.000000,1.016667,...,1.010000e+00,1.026961,6.000000,1.020000e+00,1.043361,12.000000,0.000000,69.000000,9.000000,1.000000
75%,51707.250000,683.000000,33.000000,-0.047556,-0.047556,0.000000,4.000000,4.000000,1.160000,1.146522,...,1.230000e+00,1.215575,6.000000,1.290000e+00,1.266480,12.000000,1.000000,182.000000,16.000000,4.000000
max,112586.000000,891.000000,41.000000,183.735111,182.079322,1.000000,4.000000,4.000000,851844.640000,61851.333333,...,1.513959e+06,56077.500000,6.000000,2.567408e+06,50215.555556,12.000000,1.000000,347.000000,24.000000,5.000000


In [ ]:
# 删除与交易表重复的列
duplicat_columns=list(set(new_merchant_transactions.columns)&set(merchant_info.columns))


In [34]:
merchant_info_pre=merchant_info.drop(duplicat_columns,axis=1)
merchant_info_pre['merchant_id']=merchant_info['merchant_id']

# transaction表处理

In [11]:
new_merchant_transactions.columns==historical_transactions.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True])

In [16]:
transactions_info=pd.concat([new_merchant_transactions,historical_transactions],axis=0)
del new_merchant_transactions
del historical_transactions
gc.collect()

In [38]:
category_columns=['authorized_flag', 'card_id', 'city_id', 'category_1', 
       'category_3', 'merchant_category_id', 'merchant_id', 'category_2', 'state_id',
       'subsector_id']
numeric_columns=['installments','month_lag','purchase_amount']
time_columns=['purchase_date']

assert len(category_columns)+len(numeric_columns)+len(time_columns)==len(transactions_info.columns)

In [51]:
trans_multi_le=OrdinalEncoder()
transactions_info[['authorized_flag', 'category_1','category_3']]=trans_multi_le.fit_transform(transactions_info[['authorized_flag', 'category_1','category_3']].fillna('-1'))

In [65]:
transactions_info['purchase_month']=transactions_info['purchase_date'].progress_apply(lambda x:datetime.strptime(x,'%Y-%m-%d %H:%M:%S').month)
transactions_info['purchase_day']=transactions_info['purchase_date'].progress_apply(lambda x:datetime.strptime(x,'%Y-%m-%d %H:%M:%S').day)

100%|██████████| 31075392/31075392 [04:33<00:00, 113618.71it/s]


In [ ]:
transactions_info['purchase_hours']=transactions_info['purchase_date'].apply(lambda x:datetime.strptime(x,'%Y-%m-%d %H:%M:%S').hours)